In [3]:
import requests
import numpy as np
import pandas as pd

url = "https://statsapi.web.nhl.com/api/v1/teams"

r = requests.get(url)

json_data = r.json()

df = pd.json_normalize(data = json_data, 
                       record_path = ['teams'],
                       sep = "_")
                       #meta = ['copyright']) #include this paramete if want to bring in copyright metadata

col_dict = {"id":"team_id",
            "name":"team_full_name",
            "link":"team_link",
            "abbreviation":"team_abbreviation",
            "teamName":"team_name",
            "locationName":"location_name",
            "firstYearOfPlay":"first_year_of_play",
            "shortName":"team_short_name",
            "officialSiteUrl":"official_site_url",
            "franchiseId":"franchise_id_dup",
            "active":"active",
            "venue_name":"venue_name",
            "venue_link":"venue_link",
            "venue_city":"venue_city",
            "venue_timeZone_id":"venue_time_zone_id",
            "venue_timeZone_offset":"venue_time_zone_offset",
            "venue_timeZone_tz":"venue_time_zone_tz",
            "division_id":"division_id",
            "division_name":"division_name",
            "division_nameShort":"division_name_short",
            "division_link":"division_link",
            "division_abbreviation":"division_abbreviation",
            "conference_id":"conference_id",
            "conference_name":"conference_name",
            "conference_link":"conference_link",
            "franchise_franchiseId":"franchise_id",
            "franchise_teamName":"franchise_team_name",
            "franchise_link":"franchise_link",
            "venue_id":"venue_id"}

# Rename columns
df = df.rename(columns=col_dict)
df.columns = df.columns.str.upper()

#FILL ALL NA VALUES WITH EMPTY STRING
df = df.fillna('')

#CONVERT ALL COLUMNS TO STRING
df = df.astype(str)

#CONVERT FIRST_YEAR_OF_PLAY COLUMN TO INTERGER
df['FIRST_YEAR_OF_PLAY'] = df['FIRST_YEAR_OF_PLAY'].astype('int')

#CONVERT VENUE_TIME_ZONE_OFFSET COLUMN TO INTERGER
df['VENUE_TIME_ZONE_OFFSET'] = df['VENUE_TIME_ZONE_OFFSET'].astype('int')

,TEAM_ID,TEAM_FULL_NAME,TEAM_LINK,TEAM_ABBREVIATION,TEAM_NAME,LOCATION_NAME,FIRST_YEAR_OF_PLAY,TEAM_SHORT_NAME,OFFICIAL_SITE_URL,FRANCHISE_ID_DUP,...,DIVISION_NAME_SHORT,DIVISION_LINK,DIVISION_ABBREVIATION,CONFERENCE_ID,CONFERENCE_NAME,CONFERENCE_LINK,FRANCHISE_ID,FRANCHISE_TEAM_NAME,FRANCHISE_LINK,VENUE_ID
0,1,New Jersey Devils,/api/v1/teams/1,NJD,Devils,New Jersey,1982,New Jersey,http://www.newjerseydevils.com/,23,...,Metro,/api/v1/divisions/18,M,6,Eastern,/api/v1/conferences/6,23,Devils,/api/v1/franchises/23,
1,2,New York Islanders,/api/v1/teams/2,NYI,Islanders,New York,1972,NY Islanders,http://www.newyorkislanders.com/,22,...,Metro,/api/v1/divisions/18,M,6,Eastern,/api/v1/conferences/6,22,Islanders,/api/v1/franchises/22,
2,3,New York Rangers,/api/v1/teams/3,NYR,Rangers,New York,1926,NY Rangers,http://www.newyorkrangers.com/,10,...,Metro,/api/v1/divisions/18,M,6,Eastern,/api/v1/conferences/6,10,Rangers,/api/v1/franchises/10,5054.0
3,4,Philadelphia Flyers,/api/v1/teams/4,PHI,Flyers,Philadelphia,1967,Philadelphia,http://www.philadelphiaflyers.com/,16,...,Metro,/api/v1/divisions/18,M,6,Eastern,/api/v1/conferences/6,16,Flyers,/api/v1/franchises/16,5096.0
4,5,Pittsburgh Penguins,/api/v1/teams/5,PIT,Penguins,Pittsburgh,1967,Pittsburgh,http://pittsburghpenguins.com/,17,...,Metro,/api/v1/divisions/18,M,6,Eastern,/api/v1/conferences/6,17,Penguins,/api/v1/franchises/17,5034.0


In [6]:
import cx_Oracle
import os

# Oracle database connection details
username = os.getenv('Oracle_Username')
password = os.getenv('Oracle_Password')
hostname = '10.0.0.69'
port = '1521'
service_name = 'XEPDB1'

# Construct the connection string
dsn = cx_Oracle.makedsn(hostname, port, service_name=service_name)
connection = cx_Oracle.connect(username, password, dsn)

# Create a cursor object
cursor = connection.cursor()

# Define the target table name
table_name = 'dim_team_load'

# Drop the table if it already exists
drop_query = f"DROP TABLE {table_name}"
try:
    cursor.execute(drop_query)
    print(f"Dropped table {table_name}")
except cx_Oracle.DatabaseError as e:
    error_code, error_message = e.args[0].code, e.args[0].message
    if error_code != 942:  # ORA-00942: table or view does not exist
        print(f"Error occurred while dropping table {table_name}: {error_message}")

        
# Read the SQL file
sql_file = r"C:\Users\Jeremy\Documents\nhl_analysis\02_Source_Code\table_create_scripts\dim_team_load.sql"
with open(sql_file, 'r') as file:
    create_table_query = file.read()

# Create the table
cursor.execute(create_table_query)
print(f"Created table {table_name}")

# Convert dataframe to a list of tuples
data_tuples = [tuple(row) for row in df.values]

# Prepare the INSERT query with placeholders for the columns
insert_query = f"INSERT INTO {table_name} " \
               f"({', '.join(df.columns)}) " \
               f"VALUES ({', '.join([':' + str(i+1) for i in range(len(df.columns))])})"

# Insert data into the target table
cursor.executemany(insert_query, data_tuples)
print(f"Inserted {len(data_tuples)} rows into {table_name}")

# Commit the changes
connection.commit()

# Close the cursor and the database connection
cursor.close()
connection.close()

Created table dim_team_load
Inserted 32 rows into dim_team_load
